In [7]:
import argparse
from datetime import datetime
import math
import numpy as np
import pandas as pd
import time
import torch


from fireworks import LaunchPad
from jobflow import JobStore
from jobflow.managers.fireworks import flow_to_workflow
from maggma.stores.mongolike import MongoStore
from NanoParticleTools.flows.flows import get_npmc_flow
from NanoParticleTools.inputs.nanoparticle import SphericalConstraint
import uuid

from botorch.models import SingleTaskGP, ModelListGP
from botorch import fit_gpytorch_model
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch.acquisition.monte_carlo import qExpectedImprovement
from botorch.optim import optimize_acqf

from common import seed_generator, configs

from common.utils import get_int, get_qe
from common import utils


<stdin>:1:10: fatal error: 'omp.h' file not found
#include <omp.h>
         ^~~~~~~
1 error generated.


[KeOps] Warning : omp.h header is not in the path, disabling OpenMP.
[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode


In [ ]:
from botorch.acquisition import UpperConfidenceBound
bounds = torch.tensor([[0., 0., 0., 0., 5./34.], [1., 1., 1., 1., 1.]])
candidates = recommend(train_x, train_y, best_y, bounds,1)
decode_candidates(candidates)

# submit jobs

In [ ]:
from common import configs
def submit_jobs(candidates):
    '''
    submit a recipe for simulation
    '''
    store = JobStore(DOCS_STORE, additional_stores={'trajectories': DATA_STORE})
    submitted_id = []
    candidates = candidates.tolist()
    
    for candidate in candidates:
        # iterate candidates
        dopant_specifications = []
        for spec in configs.cfg['dopant_specifications']:
            spec = list(spec)
            if 'Surface6' in spec:
                dopant_specifications.append(tuple(spec))
                break
            elif spec[1] == -1:
                spec[1] = candidate.pop(0)
                dopant_specifications.append(tuple(spec))
            else:
                dopant_specifications.append(tuple(spec))
        
        constraints = []
        for radii in configs.cfg['radius']:
            if radii == -1:
                constraints.append(SphericalConstraint(candidate.pop(0)))
            else:
                constraints.append(SphericalConstraint(radii))
                
        npmc_args = {'npmc_command': configs.cfg['npmc_command'], #NPMC
                     'num_sims': configs.cfg['num_sims'], #4
                     # 'base_seed': seed_generator.genereate(), #1000
                     'base_seed': 1000, 
                     'thread_count': configs.cfg['ncpu'],
                     #'simulation_length': 100000 #
                     'simulation_time': configs.cfg['simulation_time'] #0.01s
                     }
        spectral_kinetics_args = {'excitation_wavelength': configs.cfg['excitation_wavelength'],
                                  'excitation_power': configs.cfg['excitation_power']}

        initial_state_db_args = {'interaction_radius_bound': configs.cfg['interaction_radius_bound']} #3

        # is this being used?
        np_uuid = str(uuid.uuid4())

        for doping_seed in range(configs.cfg['num_dopant_mc']): #4
            flow = get_npmc_flow(constraints=constraints,
                                 dopant_specifications=dopant_specifications,
                                 # doping_seed=seed_generator.generate(),
                                 doping_seed = doping_seed, 
                                 spectral_kinetics_args=spectral_kinetics_args,
                                 initial_state_db_args=initial_state_db_args,
                                 npmc_args=npmc_args,
                                 output_dir=configs.cfg['output_dir']
                                 )

            wf = flow_to_workflow(flow, store=store)
            mapping = LP.add_wf(wf)
            submitted_id.append(list(mapping.values())[0])
            
        print(f"Initialized {configs.cfg['num_dopant_mc']} jobs. Submitted {len(submitted_id)}.")

    return submitted_id

In [ ]:
# credential
LP_FILE = '/Users/xiaojing/my_launchpad.yaml'
LP = LaunchPad.from_file(LP_FILE)

DOCS_STORE = MongoStore.from_launchpad_file(LP_FILE, 'test_fws_npmc')
DATA_STORE = MongoStore.from_launchpad_file(LP_FILE, 'test_docs_npmc')
LAUNCH_STORE = MongoStore.from_launchpad_file(LP_FILE, 'launches')

DATETIME_FORMAT = "%Y-%m-%dT%H:%M:%S.%f"
#fw_ids = submit_jobs(candidates)

# recommend

In [73]:
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf
from botorch.optim import optimize_acqf_discrete_local_search
def recommend(train_x, train_y, discrete_choices, n_trails, inequality_constraints ):
    single_model = SingleTaskGP(train_x, train_y)
    mll = ExactMarginalLogLikelihood(single_model.likelihood, single_model)
    fit_gpytorch_model(mll)
    
    # Expected Improvement acquisition function
    # EI = qExpectedImprovement(model = single_model, best_f = best_y)
    # Upper Confidence Bound acquisition function
    UCB = UpperConfidenceBound(single_model, beta=100)
    
    # hyperparameters are super sensitive here
    candidates, _ = optimize_acqf_discrete_local_search(acq_function = UCB,
                                                        discrete_choices = discrete_choices,
                                                        q = n_trails, 
                                                        inequality_constraints = inequality_constraints ,
                                                        num_restarts = 20, 
                                                        raw_samples = 512, 
                                # options = {'batch_limit': 5, "maxiter": 200}
                                 )
    
    return candidates

def get_data_botorch(data_file, from_cloud = True):
    if from_cloud:
        #df=pd.read_csv('../saved_data/UV_log_shuffled_10initial_test1.csv', sep=',')
        df = gcloud_utils.get_df_gspread(GSPREAD_CRED, GSPREAD_NAME)
        #df = df.drop(labels=range(1, 570), axis=0)
        my_data = df.to_numpy()
        print(f"reading data log from google sheet: {GSPREAD_NAME}!")
    else:
        my_data = np.loadtxt(data_file, delimiter=',', skiprows=1)
        print(f"reading data log from local: {data_file}!")

    # features
    train_x = torch.from_numpy(my_data[:, :7])
    # labels
    train_y = torch.from_numpy(my_data[:, 8]).unsqueeze(-1)
    # best observation
    best_y = train_y.max().item()
    
    return train_x, train_y, best_y

    
    return train_x, train_y, best_y
def encode_inputs(x_arr, x_max = 34):
    '''encode simulation input to botorch'''
    for i, arr in enumerate(x_arr):
        x_arr[i, 0] = arr[0] + arr[1]
        if arr[0] + arr[1] == 0:
            x_arr[i, 1] = 0.5
        else:
            x_arr[i, 1] = arr[0] / (arr[0] + arr[1])
        x_arr[i, 2] = arr[2] + arr[3]
        if arr[2] + arr[3] == 0:
            x_arr[i, 3] = 0.5
        else:
            x_arr[i, 3] = arr[2] / (arr[2] + arr[3])
        x_arr[i, 4] = arr[4] / x_max


def decode_candidates(x_arr, x_max = 34):
    '''decode botorch recommendation candidates for simulation'''
    for i, arr in enumerate(x_arr):
        x_arr[i, 0], x_arr[i, 1] = arr[0] * arr[1], arr[0] * (1 - arr[1])
        x_arr[i, 2], x_arr[i, 3] = arr[2] * arr[3], arr[2] * (1 - arr[3])
        x_arr[i, 4] = arr[4] * x_max

In [77]:
range_radius = np.arange(5,34,2)
conc_interval = 0.001
range_sum1 = np.linspace(0,1,int(1/conc_interval +1))
range_sum2 = np.linspace(0,1,int(1/conc_interval +1))

yb1 = torch.tensor(range_sum1)
er1 = torch.tensor(range_sum1)
tm1 = torch.tensor(range_sum1)
yb2 = torch.tensor(range_sum2)
er2 = torch.tensor(range_sum2)
tm2 = torch.tensor(range_sum1)
rdi = torch.tensor(range_radius)
discrete_choices = [yb1,er1,tm1,yb2,er2,tm2,rdi]


In [78]:
constraints = [[torch.Tensor([0,1,2]).long(),torch.Tensor([-1,-1,-1]), -1],[torch.Tensor([3,4,5]).long(),torch.Tensor([-1,-1,-1]), -1]]
    

In [75]:
init_x, init_y, best_y = get_data_botorch("../saved_data/simulation_log_YbErTm.csv", from_cloud=False)
train_x, train_y = init_x[:20], init_y[:20]
#encode_inputs(train_x)

reading data log from local: ../saved_data/simulation_log_YbErTm.csv!


In [79]:
#bounds = torch.tensor([[0., 0., 0., 0., 5], [1., 1., 1., 1., 34]])
candidates = recommend(train_x, train_y, discrete_choices,1, constraints)
candidates

tensor([[4.3200e-01, 3.1100e-01, 1.8300e-01, 3.2200e-01, 4.2400e-01, 1.3000e-02,
         2.3000e+01]], dtype=torch.float64)

In [28]:
bounds = torch.tensor([[0., 0., 0., 0., 5/34], [1., 1., 1., 1., 1]])
candidates = recommend(train_x, train_y, best_y, bounds,1)
decode_candidates(candidates)
candidates

tensor([[6.0838e-01, 9.3868e-02, 5.9734e-02, 1.4535e-02, 2.6287e+01]])